In [8]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses

In [9]:
def reconstruction_loss(x, t_decoded):
    '''Reconstruction loss for the plain VAE'''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def kl_normal_loss(qm, q_logv, pm, p_logv):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
    kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    return K.mean(K.sum(kl, axis=-1))


def get_flux_loss(m, state, state_pred):
    # state, state_pred shape (batch_size, 60, 60, 2)
    # p, p_pred shape (batch_size, 60, 60, 1)
    # k shape (batch_size, 60, 60, 1)
    
    # Only consider discrepancies in total flux, not in phases (saturation not used) 
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.in_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [10]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [11]:
def create_e2c(latent_dim, u_dim, input_shape):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)
    sampler_ = e2c_util.create_sampler()

    return encoder_, decoder_, transition_, sampler_

In [12]:
# Create plain E2C model and associated loss operations

################### case specification ######################

#     data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

#     case_name = '9w_bhp'
# case_name = '9w_bhp_rate'
case_name = '9w_ms_bhp_rate'

#     case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_1'
#     case_suffix = '_single_out_rel_3'
train_suffix = '_with_p'
model_suffix = '_flux_loss'


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n6900_dt20day_nt23_nrun300.mat'
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n2300_dt20day_nt23_nrun100.mat'

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50
u_dim = 9*2 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
hf_r.close()

num_train = state_t_train.shape[0]

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
hf_r.close()

m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
m = m.reshape(60,60,1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))



m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)


m shape is  (60, 60, 1)
m_eval shape is  (2200, 60, 60, 1)
m shape is  (6600, 60, 60, 1)


In [13]:

# Construct E2C
input_shape = (60, 60, 2)
encoder, decoder, transition, sampler = create_e2c(latent_dim, u_dim, input_shape)

xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))

zt_mean, zt_logvar = encoder(xt)
zt = sampler([zt_mean, zt_logvar])
xt_rec = decoder(zt)

zt1_mean, zt1_logvar = encoder(xt1)

# zt1_pred, zt1_mean_pred, zt1_logvar_pred = transition([zt, ut])
zt1_pred, zt1_mean_pred = transition([zt, zt_mean, ut])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1


loss_kl = kl_normal_loss(zt_mean, zt_logvar, 0., 0.)  # log(1.) = 0.
loss_bound = loss_rec_t + loss_rec_t1 + loss_kl  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1

# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)

# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = kl_normal_loss(zt1_mean_pred, zt_logvar, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans



In [14]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

iterate = K.function([xt, ut, xt1, m_tf], [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch = m[ind0:ind0 + batch_size, ...]

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, binary loss %f, binary loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval])

    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))


encoder.save_weights(output_dir + 'e2c_encoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/1650, Loss 5473.817871, Loss rec 2449.146484, loss rec t1 2935.941406, loss kl 0.011339, loss_trans 0.007562, loss flux 43.008205, loss flux t1 45.702984, binary loss 7.239711, binary loss t1 8.713845
Epoch 1/10, Batch 11/1650, Loss 1892.093140, Loss rec 807.189636, loss rec t1 947.972290, loss kl 2.134185, loss_trans 2.230043, loss flux 69.375496, loss flux t1 63.191536, binary loss 10.198698, binary loss t1 8.492787
Epoch 1/10, Batch 21/1650, Loss 1722.424561, Loss rec 763.721130, loss rec t1 564.635498, loss kl 5.849187, loss_trans 5.125171, loss flux 168.219940, loss flux t1 214.873566, binary loss 5.955356, binary loss t1 5.185890
Epoch 1/10, Batch 31/1650, Loss 995.683289, Loss rec 340.264954, loss rec t1 322.177765, loss kl 3.007895, loss_trans 2.150449, loss flux 174.745514, loss flux t1 153.336700, binary loss 8.438710, binary loss t1 7.112660
Epoch 1/10, Batch 41/1650, Loss 858.768494, Loss rec 278.871643, loss rec t1 300.319885, loss kl 1.907560, loss_tra

Epoch 1/10, Batch 381/1650, Loss 280.065338, Loss rec 105.345245, loss rec t1 113.211731, loss kl 1.258834, loss_trans 0.375491, loss flux 30.950527, loss flux t1 28.923508, binary loss 3.499990, binary loss t1 3.154669
Epoch 1/10, Batch 391/1650, Loss 296.369965, Loss rec 97.486359, loss rec t1 137.663635, loss kl 0.937136, loss_trans 0.300370, loss flux 31.686790, loss flux t1 28.295650, binary loss 4.306915, binary loss t1 3.898610
Epoch 1/10, Batch 401/1650, Loss 282.723663, Loss rec 140.346634, loss rec t1 79.576828, loss kl 1.456538, loss_trans 0.280880, loss flux 30.766171, loss flux t1 30.296602, binary loss 1.767277, binary loss t1 1.932505
Epoch 1/10, Batch 411/1650, Loss 263.492310, Loss rec 90.252975, loss rec t1 106.864563, loss kl 1.221031, loss_trans 0.339836, loss flux 32.595997, loss flux t1 32.217918, binary loss 4.928866, binary loss t1 4.127343
Epoch 1/10, Batch 421/1650, Loss 198.936508, Loss rec 67.374084, loss rec t1 80.752449, loss kl 0.468106, loss_trans 0.1732

Epoch 1/10, Batch 761/1650, Loss 127.992722, Loss rec 39.412071, loss rec t1 44.294060, loss kl 0.500482, loss_trans 0.080873, loss flux 22.855536, loss flux t1 20.849701, binary loss 3.504015, binary loss t1 3.392197
Epoch 1/10, Batch 771/1650, Loss 139.851410, Loss rec 45.538761, loss rec t1 49.583900, loss kl 1.002228, loss_trans 0.154895, loss flux 23.006504, loss flux t1 20.565130, binary loss 1.789480, binary loss t1 1.928927
Epoch 1/10, Batch 781/1650, Loss 129.382187, Loss rec 40.720596, loss rec t1 43.922573, loss kl 0.977702, loss_trans 0.145339, loss flux 23.045853, loss flux t1 20.570118, binary loss 2.306427, binary loss t1 2.067243
Epoch 1/10, Batch 791/1650, Loss 125.667389, Loss rec 36.134045, loss rec t1 45.963387, loss kl 0.754573, loss_trans 0.138612, loss flux 21.677567, loss flux t1 20.999214, binary loss 2.690424, binary loss t1 2.436908
Epoch 1/10, Batch 801/1650, Loss 112.246353, Loss rec 32.317200, loss rec t1 39.646416, loss kl 0.677024, loss_trans 0.114637, l

Epoch 1/10, Batch 1141/1650, Loss 83.724617, Loss rec 22.351004, loss rec t1 25.539413, loss kl 0.481227, loss_trans 0.079049, loss flux 18.789696, loss flux t1 16.484224, binary loss 3.145300, binary loss t1 3.190753
Epoch 1/10, Batch 1151/1650, Loss 100.813477, Loss rec 29.047863, loss rec t1 32.102661, loss kl 0.578331, loss_trans 0.081961, loss flux 19.025589, loss flux t1 19.977076, binary loss 3.488528, binary loss t1 3.226167
Epoch 1/10, Batch 1161/1650, Loss 95.192932, Loss rec 29.588448, loss rec t1 28.521118, loss kl 0.614647, loss_trans 0.083781, loss flux 19.540733, loss flux t1 16.844210, binary loss 3.067814, binary loss t1 2.732347
Epoch 1/10, Batch 1171/1650, Loss 74.538689, Loss rec 19.412073, loss rec t1 23.843678, loss kl 0.323610, loss_trans 0.049387, loss flux 15.739193, loss flux t1 15.170751, binary loss 3.336830, binary loss t1 3.050088
Epoch 1/10, Batch 1181/1650, Loss 95.887863, Loss rec 29.576256, loss rec t1 27.481579, loss kl 0.579080, loss_trans 0.079792, 

Epoch 1/10, Batch 1521/1650, Loss 63.954430, Loss rec 15.249386, loss rec t1 18.808584, loss kl 0.951796, loss_trans 0.032691, loss flux 15.442858, loss flux t1 13.469114, binary loss 2.176640, binary loss t1 2.021681
Epoch 1/10, Batch 1531/1650, Loss 80.527496, Loss rec 23.319836, loss rec t1 24.294247, loss kl 0.793335, loss_trans 0.049255, loss flux 16.997555, loss flux t1 15.073273, binary loss 1.876601, binary loss t1 1.711678
Epoch 1/10, Batch 1541/1650, Loss 77.983597, Loss rec 20.983543, loss rec t1 23.851444, loss kl 0.547309, loss_trans 0.052435, loss flux 16.842873, loss flux t1 15.705993, binary loss 2.298422, binary loss t1 1.940801
Epoch 1/10, Batch 1551/1650, Loss 80.058281, Loss rec 22.107311, loss rec t1 22.797729, loss kl 0.790607, loss_trans 0.064455, loss flux 19.013325, loss flux t1 15.284861, binary loss 1.874350, binary loss t1 1.904290
Epoch 1/10, Batch 1561/1650, Loss 66.240593, Loss rec 17.937336, loss rec t1 17.308882, loss kl 0.463288, loss_trans 0.041249, l

...

Epoch 9/10, Batch 1511/1650, Loss 14.844603, Loss rec 2.766738, loss rec t1 4.037202, loss kl 0.156932, loss_trans 0.010226, loss flux 3.766701, loss flux t1 4.106804, binary loss 1.014198, binary loss t1 1.153792
Epoch 9/10, Batch 1521/1650, Loss 16.909208, Loss rec 4.363952, loss rec t1 4.584667, loss kl 0.396694, loss_trans 0.005930, loss flux 4.155772, loss flux t1 3.402192, binary loss 0.218406, binary loss t1 0.322889
Epoch 9/10, Batch 1531/1650, Loss 18.912626, Loss rec 4.150436, loss rec t1 5.142422, loss kl 0.261559, loss_trans 0.006319, loss flux 4.929763, loss flux t1 4.422128, binary loss 0.637842, binary loss t1 0.637793
Epoch 9/10, Batch 1541/1650, Loss 17.847744, Loss rec 4.394987, loss rec t1 4.464650, loss kl 0.163737, loss_trans 0.009399, loss flux 4.426507, loss flux t1 4.388463, binary loss 3.809773, binary loss t1 3.259539
Epoch 9/10, Batch 1551/1650, Loss 19.835453, Loss rec 5.259523, loss rec t1 5.038309, loss kl 0.299519, loss_trans 0.008076, loss flux 5.442053,

Epoch 10/10, Batch 251/1650, Loss 11.341094, Loss rec 2.017873, loss rec t1 2.743089, loss kl 0.123195, loss_trans 0.010316, loss flux 3.286902, loss flux t1 3.159719, binary loss 0.905799, binary loss t1 1.018724
Epoch 10/10, Batch 261/1650, Loss 13.991951, Loss rec 2.488505, loss rec t1 3.547738, loss kl 0.248376, loss_trans 0.005531, loss flux 3.910929, loss flux t1 3.790871, binary loss 1.485888, binary loss t1 1.457091
Epoch 10/10, Batch 271/1650, Loss 12.150495, Loss rec 2.331918, loss rec t1 2.883193, loss kl 0.115738, loss_trans 0.010750, loss flux 3.462430, loss flux t1 3.346464, binary loss 1.705084, binary loss t1 1.718308
Epoch 10/10, Batch 281/1650, Loss 14.963619, Loss rec 3.588295, loss rec t1 3.680867, loss kl 0.167710, loss_trans 0.009716, loss flux 4.028155, loss flux t1 3.488876, binary loss 0.782096, binary loss t1 0.639339
Epoch 10/10, Batch 291/1650, Loss 13.957087, Loss rec 2.879535, loss rec t1 3.374602, loss kl 0.165200, loss_trans 0.011481, loss flux 3.702626,

Epoch 10/10, Batch 641/1650, Loss 24.128126, Loss rec 7.291988, loss rec t1 6.656064, loss kl 0.165045, loss_trans 0.014396, loss flux 5.298948, loss flux t1 4.701685, binary loss 2.445850, binary loss t1 2.150154
Epoch 10/10, Batch 651/1650, Loss 23.507538, Loss rec 5.162746, loss rec t1 5.848024, loss kl 0.145300, loss_trans 0.010244, loss flux 6.892759, loss flux t1 5.448463, binary loss 0.937954, binary loss t1 1.012082
Epoch 10/10, Batch 661/1650, Loss 18.617565, Loss rec 3.716326, loss rec t1 5.639498, loss kl 0.133613, loss_trans 0.013559, loss flux 4.516153, loss flux t1 4.598414, binary loss 2.984915, binary loss t1 2.702553
Epoch 10/10, Batch 671/1650, Loss 27.430681, Loss rec 7.788765, loss rec t1 8.094218, loss kl 0.191173, loss_trans 0.012785, loss flux 6.254247, loss flux t1 5.089493, binary loss 1.420617, binary loss t1 1.177102
Epoch 10/10, Batch 681/1650, Loss 19.041945, Loss rec 5.017280, loss rec t1 5.598642, loss kl 0.127844, loss_trans 0.015830, loss flux 4.115578,

Epoch 10/10, Batch 1031/1650, Loss 16.460220, Loss rec 3.999763, loss rec t1 4.813927, loss kl 0.138003, loss_trans 0.005887, loss flux 3.725510, loss flux t1 3.777129, binary loss 1.244453, binary loss t1 1.433915
Epoch 10/10, Batch 1041/1650, Loss 15.608537, Loss rec 3.396394, loss rec t1 4.580492, loss kl 0.141935, loss_trans 0.009841, loss flux 3.701117, loss flux t1 3.778757, binary loss 2.142295, binary loss t1 2.054869
Epoch 10/10, Batch 1051/1650, Loss 17.443575, Loss rec 4.609339, loss rec t1 4.253437, loss kl 0.226899, loss_trans 0.010941, loss flux 4.604427, loss flux t1 3.738533, binary loss 1.490390, binary loss t1 1.285696
Epoch 10/10, Batch 1061/1650, Loss 12.800418, Loss rec 2.750073, loss rec t1 3.021726, loss kl 0.124929, loss_trans 0.008127, loss flux 3.500449, loss flux t1 3.395114, binary loss 1.670885, binary loss t1 1.740633
Epoch 10/10, Batch 1071/1650, Loss 16.199741, Loss rec 3.183048, loss rec t1 4.365814, loss kl 0.309115, loss_trans 0.007310, loss flux 4.18

Epoch 10/10, Batch 1421/1650, Loss 18.618324, Loss rec 4.098661, loss rec t1 4.385388, loss kl 0.136104, loss_trans 0.005512, loss flux 5.191366, loss flux t1 4.801292, binary loss 1.887794, binary loss t1 1.655398
Epoch 10/10, Batch 1431/1650, Loss 19.370661, Loss rec 6.191673, loss rec t1 4.429347, loss kl 0.422994, loss_trans 0.011117, loss flux 4.363784, loss flux t1 3.951748, binary loss 4.074263, binary loss t1 2.387027
Epoch 10/10, Batch 1441/1650, Loss 18.902538, Loss rec 3.659836, loss rec t1 4.272242, loss kl 0.231114, loss_trans 0.007240, loss flux 5.422732, loss flux t1 5.309373, binary loss 1.170398, binary loss t1 0.982202
Epoch 10/10, Batch 1451/1650, Loss 16.928625, Loss rec 3.639842, loss rec t1 4.148340, loss kl 0.155167, loss_trans 0.011930, loss flux 4.817894, loss flux t1 4.155452, binary loss 1.093947, binary loss t1 1.097353
Epoch 10/10, Batch 1461/1650, Loss 13.251588, Loss rec 2.639066, loss rec t1 2.515431, loss kl 0.164861, loss_trans 0.006304, loss flux 4.13